In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text as text

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [13]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
df = pd.read_csv('./datasets/dataset.csv')
df['corpus'] = df['corpus'].astype(str)
df

,corpus
0,"A more precise measurement, such as a urine al..."
1,What are the symptoms of Fraser like syndrome ...
2,Most cases of shingles occur in adults. Only a...
3,Is Bilateral perisylvian polymicrogyria inheri...
4,""",GARD,Coccygodynia What are the treatments fo..."
...,...
77802,"In the fatal cases (4/5 patients), a transient..."
77803,This chemical robs your blood of oxygen and tr...
77804,Physical Activity Being physically active and ...
77805,""",GARD,Faciocardiorenal syndrome What is (are)..."


In [15]:
samples = df['corpus'].values
samples[0]

'A more precise measurement, such as a urine albumin-to-creatinine ratio, may be necessary to confirm kidney disease. Unlike a dipstick test for albumin, a urine albumin-to-creatinine ratiothe ratio between the amount of albumin and the amount of creatinine in urineis not affected by variation in urine concentration. Blood test.'

In [16]:
dataset = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(samples))

for txt in dataset.take(1):
    print("Sentence: ", txt.numpy().decode('utf-8'))

Sentence:  A more precise measurement, such as a urine albumin-to-creatinine ratio, may be necessary to confirm kidney disease. Unlike a dipstick test for albumin, a urine albumin-to-creatinine ratiothe ratio between the amount of albumin and the amount of creatinine in urineis not affected by variation in urine concentration. Blood test.


In [17]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [18]:
vocab = bert_vocab.bert_vocab_from_dataset(
    dataset,
    **bert_vocab_args
)

In [19]:
with open('vocab.txt', 'w', encoding='utf-8') as file:
    for token in vocab:
        print(token, file=file)

In [20]:
len(vocab)

7951

In [30]:
tokenizer = text.BertTokenizer('vocab.txt', **bert_tokenizer_params)

In [31]:
BUFFER_SIZE = 20000
BATCH_SIZE = 32
MAX_TOKENS = 129

def prepare_data(token):
      token = token[:MAX_TOKENS]
      input = token[:-1]
      label = token[1:]
      
      return input, label

train_dataset = tokenizer.tokenize(samples[:166021]).merge_dims(-2,-1).to_tensor()
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
train_dataset = train_dataset.map(prepare_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

val_dataset = tokenizer.tokenize(samples[166021:]).merge_dims(-2,-1).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices(val_dataset)
val_dataset = val_dataset.map(prepare_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

for x_train, y_train in train_dataset.take(1):
      break

print(x_train.shape)
print(y_train.shape)

print('Input:', x_train[0][:15].numpy())
print('Output:', y_train[0][:15].numpy())


(32, 128)
(32, 128)
Input: [ 144  156  649   96   97  271  291   86   85 1437   11  792 3422   12
   17]
Output: [ 156  649   96   97  271  291   86   85 1437   11  792 3422   12   17
  216]


In [32]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

In [33]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [34]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [35]:
class CausalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x,
            use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [36]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [37]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()

        self.causal_self_attention = CausalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.causal_self_attention(x=x)
        x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
        return x

In [38]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size, d_model=d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_layers = [
            DecoderLayer(d_model=d_model, num_heads=num_heads,
                        dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

        self.last_attn_scores = None

    def call(self, x):
        # `x` is token-IDs shape (batch, target_seq_len)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.dec_layers[i](x)

        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [39]:
class Transformer(tf.keras.Model):
    def __init__(self, *, num_layers, d_model, num_heads, dff, target_vocab_size, dropout_rate=0.1):
        super().__init__()

        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=dff,
                            vocab_size=target_vocab_size,
                            dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, x):
        # To use a Keras model with `.fit` you must pass all your inputs in the
        # first argument.

        x = self.decoder(x)  # (batch_size, target_len, d_model)

        # Final linear layer output.
        logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

        # Return the final output and the attention weights.
        return logits

In [40]:
num_layers = 2
d_model = 64
dff = 512
num_heads = 8
dropout_rate = 0.1

In [41]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    target_vocab_size=len(vocab),
    dropout_rate=dropout_rate)

output = transformer(x_train)

print(x_train.shape)
print(output.shape)

(32, 128)
(32, 128, 7735)


In [42]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder (Decoder)           multiple                  893120    
                                                                 
 dense_4 (Dense)             multiple                  502775    
                                                                 
Total params: 1,395,895
Trainable params: 1,395,895
Non-trainable params: 0
_________________________________________________________________


In [43]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [44]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [45]:
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = loss_object(label, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss


def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [46]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_masked_accuracy', patience=3)
transformer.compile(loss=masked_loss, optimizer=optimizer, metrics=[masked_accuracy])

In [47]:
transformer.fit(train_dataset, epochs=20, validation_data=val_dataset, callbacks=[es_callback])

Epoch 1/20
5189/5189 [==============================] - 799s 153ms/step - loss: 3.4576 - masked_accuracy: 0.3349 - val_loss: 2.4372 - val_masked_accuracy: 0.4610
Epoch 2/20
5189/5189 [==============================] - 737s 142ms/step - loss: 2.3977 - masked_accuracy: 0.4598 - val_loss: 2.2214 - val_masked_accuracy: 0.4952
Epoch 3/20
5189/5189 [==============================] - 690s 133ms/step - loss: 2.2850 - masked_accuracy: 0.4772 - val_loss: 2.1511 - val_masked_accuracy: 0.5069
Epoch 4/20
5189/5189 [==============================] - 688s 133ms/step - loss: 2.2381 - masked_accuracy: 0.4850 - val_loss: 2.1165 - val_masked_accuracy: 0.5128
Epoch 5/20
5189/5189 [==============================] - 686s 132ms/step - loss: 2.2106 - masked_accuracy: 0.4898 - val_loss: 2.0928 - val_masked_accuracy: 0.5169
Epoch 6/20
5189/5189 [==============================] - 688s 133ms/step - loss: 2.1914 - masked_accuracy: 0.4932 - val_loss: 2.0728 - val_masked_accuracy: 0.5212
Epoch 7/20
5189/5189 [======

In [58]:
transformer.save_weights('./training_1/checkpoints')

In [59]:
# Loads the weights
transformer.load_weights('./training_1/checkpoints')

In [60]:
def generate_text(sentence, maxlen=MAX_TOKENS):
    output_array = tokenizer.tokenize(sentence).merge_dims(-2, -1).to_tensor()
    
    for i in range(maxlen):
        prediction = transformer(output_array, training=False)
        prediction = prediction[:, -1:, :]
        prediction = tf.argmax(prediction, axis=-1)
        output_array = tf.concat([output_array, prediction], axis=1)
        
        if prediction[0][0].numpy() == 3:
            break
        
    output = tokenizer.detokenize(output_array).to_tensor()
    output = ' '.join([word.decode('utf-8') for word in output.numpy()[0]])
    return output

In [61]:
generate_text('what is type 2 diabetes')

"what is type 2 diabetes ? diabetes is a condition in which the body ' s cells are able to produce insulin . when the body ' s cells are able to use insulin , the body ' s cells , the body ' s cells , and the cells are able to use insulin . insulin resistance is a condition in which the body ' s cells are able to use insulin effectively . insulin resistance is a condition in which the body ' s cells are able to use insulin effectively . insulin resistance is a condition in which the body ' s cells are able to use insulin effectively . insulin resistance is a hormone that helps control insulin production . insulin resistance is a hormone that is a hormone that"

In [62]:
generate_text('characteristic of X-linked inheritance')

'characteristic of x - linked inheritance is that fathers cannot pass x - linked traits to their sons . " , ghr , x - linked traits to x - linked traits to their sons what are the treatments for x - linked inheritance ? , these resources address the diagnosis or management of x - linked recessive inheritance : - gene review : gene review : x - linked recessive inheritance - gene review : gene review : x - linked recessive disorders - genetic testing registry : x - linked recessive disorders - medlineplus encyclopedia : x - linked recessive disorders these resources from medlineplus offer information about the diagnosis and management of various health conditions : - diagnostic tests - drug therapy - surgery and rehabilitation - genetic counseling - palliative care'